<a href="https://colab.research.google.com/github/manumeehl/trajectories/blob/main/trajectories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install tesseract-ocr
!pip install opencv-python pytesseract pytube

In [10]:
from google.colab.patches import cv2_imshow as show
from pytube import YouTube
import pytesseract
import cv2
import re

In [14]:
def get_video(url, path):
  vid = YouTube(url)
  print(f"Downloading Video '{vid.streams[0].title}'")
  vid.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first().download(output_path=path)
  print('Download complete')

In [3]:
def custom_crop(frame):
  height, width, _ = frame.shape
  return frame[height-140:height,0:1920]

In [4]:
def extract(string):
  ''' Extract mission clock, altitude and velocity from a given string '''
  string = string.lower()

  # Apply a blacklist
  blacklist = ['stage 1', 'stage 2', 'falcon 9']
  for w in blacklist:
    string = string.replace(w, '')

  # Find all numbers and floats in the string
  numbers = re.findall(r'\d+(?:\.\d+)?', string)

  # Find the mission clock (T+00:00:00)
  mission_clock = re.findall(r't\+\d{2}:\d{2}:\d{2}', string)

  # Remove numbers which also occur in the mission clock
  mission_clock_numbers = re.findall(r'\d+(?:\.\d+)?', ' '.join(mission_clock))
  for num in mission_clock_numbers:
      numbers.remove(num)

  # Remove duplicates from the numbers list
  numbers = list(set(numbers))

  # Sort the numbers list in ascending order
  numbers.sort()

  # Return the sorted numbers and mission clock
  return numbers, mission_clock

In [ ]:
for i in range(3):
  frame = cv2.imread(f'frames/frame_{i+1}.png')
  crop = custom_crop(frame)
  gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
  inverted = cv2.bitwise_not(gray)
  show(inverted)

  output = pytesseract.image_to_string(inverted)
  print(extract(output))

In [15]:
url = 'https://www.youtube.com/watch?v=agYuEAkEljw'
get_video(url, 'launches')

Download complete


In [ ]:
vidpath = 'launches/Starlink_Mission.mp4'
vid = cv2.VideoCapture(vidpath)
fps = vid.get(cv2.CAP_PROP_FPS)

print('Awaiting liftoff')
